### DATA CLEANING ###

Remove of the columns we are not goin to use

In [1]:
import pandas as pd


df = pd.read_csv('Assignment DataSet.csv')

#df.info()


In [2]:
columns_to_remove = [0, 1, 5, 6, 7, 18, 20, 22, 24, 27, 28, 29, 32, 33, 34] # List of column positions not interesting for the work
df = df.drop(df.columns[columns_to_remove], axis=1)

new_file_name = 'modified_file.csv'
df.to_csv(new_file_name, index=False)

In [11]:
import pandas as pd
import numpy as np

df = pd.read_csv('modified_file.csv')
duplicate_rows = df.duplicated()

counter=duplicate_rows.sum()

print(f'Number of duplicated rows: {counter}')

df.head()

Number of duplicated rows: 0


,race,gender,age,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,...,number_diagnoses,glimepiride,glyburide,pioglitazone,rosiglitazone,citoglipton,insulin,change,diabetesMed,readmitted
0,Caucasian,Female,[80-90),13,69,4,38,0,0,1,...,9,No,No,No,No,Steady,Steady,No,Yes,NO
1,Caucasian,Female,[80-90),5,34,0,4,0,0,0,...,7,No,No,No,No,Steady,No,No,No,NO
2,Other,Male,[60-70),1,37,1,13,3,0,0,...,5,No,No,No,No,Steady,Steady,No,Yes,NO
3,Caucasian,Male,[40-50),4,56,2,22,0,0,0,...,9,No,No,No,No,Steady,Up,Ch,Yes,YES
4,Caucasian,Female,[70-80),1,38,2,1,0,1,2,...,7,No,No,No,No,Steady,No,No,No,NO


Let's check the autocorrelation in the numerical to see if their value are due to randomness

In [4]:
def autocorr(x):
    result = np.correlate(x, x, mode='full')
    return result[result.size // 2:]

Substitute '?' values with a random nominal value, with the same frequency distribution

In [12]:
freq_dist = df['race'][df['race'] != '?'].value_counts(normalize=True)

nan_val=df['race'][df['race'] == '?'].value_counts()

cum_prob_dist = freq_dist.cumsum()
        
random_values = np.random.rand(sum(df['race'] == '?'))
df['race'] = df['race'].apply(lambda x: freq_dist.idxmax() if x == '?' else x)
df.head()


,race,gender,age,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,...,number_diagnoses,glimepiride,glyburide,pioglitazone,rosiglitazone,citoglipton,insulin,change,diabetesMed,readmitted
0,Caucasian,Female,[80-90),13,69,4,38,0,0,1,...,9,No,No,No,No,Steady,Steady,No,Yes,NO
1,Caucasian,Female,[80-90),5,34,0,4,0,0,0,...,7,No,No,No,No,Steady,No,No,No,NO
2,Other,Male,[60-70),1,37,1,13,3,0,0,...,5,No,No,No,No,Steady,Steady,No,Yes,NO
3,Caucasian,Male,[40-50),4,56,2,22,0,0,0,...,9,No,No,No,No,Steady,Up,Ch,Yes,YES
4,Caucasian,Female,[70-80),1,38,2,1,0,1,2,...,7,No,No,No,No,Steady,No,No,No,NO


### CONVERT NOMINAL COLUMNS INTO NUMERICAL ###

In [22]:
df_copy_numerical = df.copy()
nominal_columns=[0,1,2,14,15,16,17,18,19,20,21]
for j in nominal_columns:
    column_race = df_copy_numerical[df_copy_numerical.columns[j]].value_counts()
    map_dict={}
    count_race=0
    for value in column_race.index:
        map_dict[value]=count_race
        count_race+=1

    df_copy_numerical[df_copy_numerical.columns[j]] = df_copy_numerical[df_copy_numerical.columns[j]].map(map_dict)
    df_copy_numerical[df_copy_numerical.columns[j]] = df_copy_numerical[df_copy_numerical.columns[j]].astype(int)

df_copy_numerical.head()

,race,gender,age,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,...,number_diagnoses,glimepiride,glyburide,pioglitazone,rosiglitazone,citoglipton,insulin,change,diabetesMed,readmitted
0,0,0,2,13,69,4,38,0,0,1,...,9,0,0,0,0,0,1,0,0,NO
1,0,0,2,5,34,0,4,0,0,0,...,7,0,0,0,0,0,0,0,1,NO
2,3,1,1,1,37,1,13,3,0,0,...,5,0,0,0,0,0,1,0,0,NO
3,0,1,4,4,56,2,22,0,0,0,...,9,0,0,0,0,0,3,1,0,YES
4,0,0,0,1,38,2,1,0,1,2,...,7,0,0,0,0,0,0,0,1,NO


### START OF THE ANALYSIS ###

In [23]:
df_copy_numerical.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   race                5000 non-null   int32  
 1   gender              5000 non-null   int32  
 2   age                 5000 non-null   int32  
 3   time_in_hospital    5000 non-null   int64  
 4   num_lab_procedures  5000 non-null   int64  
 5   num_procedures      5000 non-null   int64  
 6   num_medications     5000 non-null   int64  
 7   number_outpatient   5000 non-null   int64  
 8   number_emergency    5000 non-null   int64  
 9   number_inpatient    5000 non-null   int64  
 10  diag_1              5000 non-null   float64
 11  diag_2              5000 non-null   float64
 12  diag_3              5000 non-null   float64
 13  number_diagnoses    5000 non-null   int64  
 14  glimepiride         5000 non-null   int32  
 15  glyburide           5000 non-null   int32  
 16  piogli

### KNN ###

In [24]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold

# to use KNN we can only use numerical data
dFeat = df_copy_numerical.iloc[:, 0:21]
dClass=df_copy_numerical.iloc[:,-1]

kf=KFold(n_splits=2000)
score=[]

for train, test in kf.split(dFeat,dClass):
    dClass_training=dClass.iloc[train]
    dClass_test=dClass.iloc[test]
    dFeat_training=dFeat.iloc[train,:]
    dFeat_test=dFeat.iloc[test,:]

    model = KNeighborsClassifier(n_neighbors=3)
    model.fit(dFeat_training,dClass_training)
    score.append(model.score(dFeat_test, dClass_test))
print(f'KNN result with cross validation: {np.mean(score)}')

# 0.5125833333333334

KNN result with cross validation: 0.5125833333333334


### DECISION TREE ###

In [25]:
from sklearn.tree import DecisionTreeClassifier

dFeat = df_copy_numerical.iloc[:, 0:21]
dClass=df_copy_numerical.iloc[:,-1]

kf=KFold(n_splits=10)
score=[]
for train, test in kf.split(dFeat,dClass):
    dClass_training=dClass.iloc[train]
    dClass_test=dClass.iloc[test]
    dFeat_training=dFeat.iloc[train,:]
    dFeat_test=dFeat.iloc[test,:]

    model = DecisionTreeClassifier()
    model.fit(dFeat_training,dClass_training)
    score.append(model.score(dFeat_test, dClass_test))

print(f'Decision tree result with cross validation: {np.mean(score)}')

# 0.5446


Decision tree result with cross validation: 0.5418000000000001


### RANDOM FOREST ###

In [26]:
from sklearn.ensemble import RandomForestClassifier

dFeat = df_copy_numerical.iloc[:, 0:21]
dClass=df_copy_numerical.iloc[:,-1]

kf=KFold(n_splits=10)
score=[]
for train, test in kf.split(dFeat,dClass):
    dClass_training=dClass.iloc[train]
    dClass_test=dClass.iloc[test]
    dFeat_training=dFeat.iloc[train,:]
    dFeat_test=dFeat.iloc[test,:]

    model = RandomForestClassifier(n_estimators=1000, min_samples_split=10)
    model.fit(dFeat_training,dClass_training)
    score.append(model.score(dFeat_test, dClass_test))

print(f'RF result with cross validation: {np.mean(score)}')

# 0.6237999999999999

RF result with cross validation: 0.626
